In [11]:
import numpy as np
import pandas as pd
import re

### useful funcs

In [72]:
def find_words(str):
    pattern = r'\b[A-Za-z\']+\b'
    full_words = re.findall(pattern, str)
    return full_words

### loading data and processing

In [73]:
#loading in data and getting list of words per record
review_train_data = pd.read_csv("movie_review.csv",encoding= 'unicode_escape')
review_train_data['words'] = review_train_data['Review'].apply(find_words)
review_train_data

,Review,Polarity,words
0,summer catch ( 2001 ) . starring freddie prinz...,-1,"[summer, catch, starring, freddie, prinze, jr,..."
1,as you should know this summer has been less ...,-1,"[as, you, should, know, this, summer, has, bee..."
2,back in the early years of his career eddie m...,-1,"[back, in, the, early, years, of, his, career,..."
3,the modern use of the western movie and the w...,-1,"[the, modern, use, of, the, western, movie, an..."
4,note : some may consider portions of the follo...,-1,"[note, some, may, consider, portions, of, the,..."
...,...,...,...
195,er is een leuke vondst in deze film . in het a...,1,"[er, is, een, leuke, vondst, in, deze, film, i..."
196,steven spielberg is now considered as one of t...,1,"[steven, spielberg, is, now, considered, as, o..."
197,member of the 'internet movie critic's associa...,1,"[member, of, the, internet, movie, critic's, a..."
198,after watching the first ten minutes of this j...,1,"[after, watching, the, first, ten, minutes, of..."


In [74]:
#get all negative words and positive words 
train_pos = review_train_data[review_train_data["Polarity"] == 1]['words'].to_numpy().flatten()

In [75]:
train_pos

array([list(['the', 'farrelly', 'brothers', 'third', 'film', 'after', 'dumb', 'and', 'dumber', 'and', 'kingpin', 'brings', 'together', 'the', 'real', 'life', 'couple', 'of', 'cameron', 'diaz', 'and', 'matt', 'dillon', 'some', 'nasty', 'humour', 'a', 'cute', 'dog', 'and', 'a', 'mix', 'of', 'love', 'fate', 'and', 'romance', 'plot', 'high', 'school', 'nerd', 'ted', 'gets', 'lucky', 'when', 'the', 'cutest', 'girl', 'in', 'his', 'class', 'asks', 'him', 'to', 'the', 'prom', 'unfortunately', 'for', 'ted', 'he', 'accidentally', 'gets', 'a', 'part', 'of', 'his', 'male', 'anatomy', 'ouch', 'caught', 'in', 'his', 'zipper', 'and', 'misses', 'the', 'big', 'night', 'thirteen', 'years', 'later', 'ted', 'still', "can't", 'get', 'mary', 'out', 'of', 'his', 'mind', 'and', 'hires', 'a', 'private', 'detective', 'dillon', 'to', 'find', 'her', 'once', 'found', 'it', 'turns', 'out', 'that', 'mary', 'has', 'more', 'than', 'one', 'suitor', 'and', "that's", 'when', 'the', 'real', 'fun', 'begins', 'critique', 'l

In [70]:

# Example 2D array
two_d_array = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

# Flatten using numpy.flatten
flattened_array = two_d_array.flatten()

print(flattened_array)


[1 2 3 4 5 6 7 8 9]


In [71]:
two_d_array

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])